# Phát triển mô hình phân loại ảnh động vật bằng CNN

Bước 1: Kiểm tra và cài đặt các thư viện cần thiết

In [1]:
# 1. Import các thư viện cần thiết
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import zipfile
import shutil
import random
import warnings
warnings.filterwarnings('ignore')

Bước 2: Kiểm tra dữ liệu đầu vào

Ở bước này, bạn cần kiểm tra xem thư mục chứa dữ liệu (các file zip ảnh động vật) đã có đầy đủ chưa. Nếu thiếu file nào, bạn cần bổ sung trước khi tiếp tục. Việc kiểm tra này giúp đảm bảo pipeline không bị lỗi do thiếu dữ liệu.

In [2]:
# 2. Kiểm tra dữ liệu đầu vào
import os

def check_data_structure(animals_dir='./animals'):
    """Kiểm tra thư mục và file dữ liệu zip động vật"""
    expected_files = [
        'butterfly.zip', 'cat.zip', 'chicken.zip', 'cow.zip', 'dog.zip',
        'elephant.zip', 'horse.zip', 'sheep.zip', 'spider.zip', 'squirrel.zip'
    ]
    if not os.path.exists(animals_dir):
        print(f"✗ Không tìm thấy thư mục {animals_dir}")
        return False
    missing_files = []
    for file in expected_files:
        file_path = os.path.join(animals_dir, file)
        if os.path.exists(file_path):
            print(f"✓ {file}")
        else:
            missing_files.append(file)
            print(f"✗ {file}")
    if missing_files:
        print(f"\nThiếu {len(missing_files)} file zip trong thư mục animals/")
        return False
    print("\nDữ liệu đầu vào đã sẵn sàng!")
    return True

check_data_structure()

✓ butterfly.zip
✓ cat.zip
✓ chicken.zip
✓ cow.zip
✓ dog.zip
✓ elephant.zip
✓ horse.zip
✓ sheep.zip
✓ spider.zip
✓ squirrel.zip

Dữ liệu đầu vào đã sẵn sàng!


True

Bước 3: Giải nén và tổ chức lại dữ liệu

Sau khi kiểm tra dữ liệu, bạn cần giải nén các file zip và tổ chức lại thành hai thư mục `train` và `validation` cho từng lớp. Điều này giúp việc huấn luyện và đánh giá mô hình thuận tiện, đúng chuẩn với các thư viện xử lý ảnh của Keras.

In [3]:
# 3. Giải nén và tổ chức lại dữ liệu thành train/validation
import zipfile
import shutil
import random

def extract_data(animals_dir='./animals', extract_path='./data'):
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    zip_files = [
        'butterfly.zip', 'cat.zip', 'chicken.zip', 'cow.zip', 'dog.zip', 
        'elephant.zip', 'horse.zip', 'sheep.zip', 'spider.zip', 'squirrel.zip'
    ]
    for zip_file in zip_files:
        zip_path = os.path.join(animals_dir, zip_file)
        if os.path.exists(zip_path):
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            print(f"Đã giải nén: {zip_file}")
        else:
            print(f"Không tìm thấy file: {zip_path}")

def organize_data(data_path='./data'):
    train_dir = os.path.join(data_path, 'train')
    val_dir = os.path.join(data_path, 'validation')
    if os.path.exists(train_dir):
        shutil.rmtree(train_dir)
    if os.path.exists(val_dir):
        shutil.rmtree(val_dir)
    os.makedirs(train_dir)
    os.makedirs(val_dir)
    for item in os.listdir(data_path):
        item_path = os.path.join(data_path, item)
        if item in ['train', 'validation'] or not os.path.isdir(item_path):
            continue
        class_name = item.lower()
        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)
        images = []
        for root, dirs, files in os.walk(item_path):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    images.append(os.path.join(root, file))
        random.shuffle(images)
        split_idx = int(0.8 * len(images))
        train_images = images[:split_idx]
        val_images = images[split_idx:]
        for i, img_path in enumerate(train_images):
            ext = os.path.splitext(img_path)[1]
            new_name = f"{class_name}_train_{i:04d}{ext}"
            shutil.copy2(img_path, os.path.join(train_class_dir, new_name))
        for i, img_path in enumerate(val_images):
            ext = os.path.splitext(img_path)[1]
            new_name = f"{class_name}_val_{i:04d}{ext}"
            shutil.copy2(img_path, os.path.join(val_class_dir, new_name))
        print(f"Lớp {class_name}: {len(train_images)} train, {len(val_images)} val")

# Thực hiện giải nén và tổ chức dữ liệu
extract_data()
organize_data()

Đã giải nén: butterfly.zip
Đã giải nén: cat.zip
Đã giải nén: cat.zip
Đã giải nén: chicken.zip
Đã giải nén: chicken.zip
Đã giải nén: cow.zip
Đã giải nén: cow.zip
Đã giải nén: dog.zip
Đã giải nén: dog.zip
Đã giải nén: elephant.zip
Đã giải nén: elephant.zip
Đã giải nén: horse.zip
Đã giải nén: horse.zip
Đã giải nén: sheep.zip
Đã giải nén: sheep.zip
Đã giải nén: spider.zip
Đã giải nén: spider.zip
Đã giải nén: squirrel.zip
Đã giải nén: squirrel.zip
Lớp cat: 1334 train, 334 val
Lớp cat: 1334 train, 334 val
Lớp butterfly: 1689 train, 423 val
Lớp butterfly: 1689 train, 423 val
Lớp dog: 3890 train, 973 val
Lớp dog: 3890 train, 973 val
Lớp sheep: 1456 train, 364 val
Lớp sheep: 1456 train, 364 val
Lớp spider: 3856 train, 965 val
Lớp spider: 3856 train, 965 val
Lớp chicken: 2478 train, 620 val
Lớp chicken: 2478 train, 620 val
Lớp horse: 2098 train, 525 val
Lớp horse: 2098 train, 525 val
Lớp squirrel: 1489 train, 373 val
Lớp squirrel: 1489 train, 373 val
Lớp cow: 1492 train, 374 val
Lớp cow: 1492 tr

Bước 4: Tạo data generators cho train/validation

Tiếp theo, bạn sẽ tạo các data generator để tự động đọc ảnh, thực hiện augmentation cho tập train và chuẩn hóa dữ liệu cho cả train/validation. Điều này giúp mô hình học tốt hơn và giảm overfitting.

In [4]:
# 4. Tạo data generators cho train/validation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
BATCH_SIZE = 32

def create_data_generators(data_path='./data'):
    train_dir = os.path.join(data_path, 'train')
    val_dir = os.path.join(data_path, 'validation')
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=25,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.15,
        brightness_range=[0.8, 1.2],
        fill_mode='nearest'
    )
    val_datagen = ImageDataGenerator(rescale=1./255)
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    return train_generator, validation_generator

train_gen, val_gen = create_data_generators()
print(f"Training samples: {train_gen.samples}")
print(f"Validation samples: {val_gen.samples}")
print(f"Classes: {list(train_gen.class_indices.keys())}")

Found 20938 images belonging to 10 classes.
Found 5241 images belonging to 10 classes.
Training samples: 20938
Validation samples: 5241
Classes: ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']
Found 5241 images belonging to 10 classes.
Training samples: 20938
Validation samples: 5241
Classes: ['butterfly', 'cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']


Bước 5: Xây dựng mô hình CNN nâng cao

Ở bước này, bạn sẽ xây dựng một mô hình CNN nâng cao với residual connections (kết nối tắt) để tăng khả năng học đặc trưng và cải thiện hiệu quả phân loại ảnh.

In [5]:
# 5. Xây dựng mô hình CNN nâng cao với residual connections
from tensorflow.keras import layers, models

def create_advanced_cnn_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = layers.Conv2D(64, (7, 7), strides=2, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)
    def residual_block(x, filters, kernel_size=3, stride=1):
        shortcut = x
        x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters, kernel_size, padding='same')(x)
        x = layers.BatchNormalization()(x)
        if stride != 1 or shortcut.shape[-1] != filters:
            shortcut = layers.Conv2D(filters, 1, strides=stride, padding='same')(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)
        x = layers.Add()([x, shortcut])
        x = layers.Activation('relu')(x)
        return x
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = layers.Dropout(0.2)(x)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    x = layers.Dropout(0.2)(x)
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)
    x = layers.Dropout(0.3)(x)
    x = residual_block(x, 512, stride=2)
    x = residual_block(x, 512)
    x = layers.Dropout(0.3)(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(train_gen.num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs, name='custom_residual_cnn')
    return model

model = create_advanced_cnn_model(train_gen.num_classes)
model.summary()

Model: "custom_residual_cnn"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 112, 112,  │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 112, 112,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 56, 56,    │     36,928 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 56, 56,    │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 56, 56,    │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 56, 56,    │     36,928 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 56, 56,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 56, 56,    │     36,928 │ activation_2[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 56, 56,    │          0 │ batch_normalizat

 Total params: 12,250,250 (46.73 MB)

 Trainable params: 12,238,602 (46.69 MB)

 Non-trainable params: 11,648 (45.50 KB)

Bước 6: Huấn luyện mô hình (5 epoch)

Tiến hành huấn luyện mô hình với tập train và kiểm tra hiệu quả trên tập validation. Số epoch đặt là 5 để kiểm tra nhanh khả năng hoạt động của pipeline và mô hình.

In [ ]:
# 6. Huấn luyện mô hình (5 epoch để test khả năng tạo mô hình)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

EPOCHS = 5

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)
]

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=callbacks,
    verbose=1
)

Bước 7: Vẽ biểu đồ quá trình huấn luyện

Sau khi huấn luyện xong, bạn sẽ vẽ biểu đồ để trực quan hóa quá trình học của mô hình (độ chính xác và loss trên cả train/validation theo từng epoch). Điều này giúp bạn đánh giá nhanh hiệu quả và xu hướng học của mô hình.

In [ ]:
# 7. Vẽ biểu đồ quá trình huấn luyện
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(history.history['accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.set_title('Model Accuracy')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True)

ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.set_title('Model Loss')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()